# Self-evaluation

In [1]:
! nvidia-smi

Thu Sep  4 13:15:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0 Off |                  Off |
|  0%   51C    P8             23W /  450W |      15MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import time
import textworld
import textworld.gym
import numpy as np
import re
import os
from glob import glob
from typing import Mapping, Any

In [3]:
import torch
import accelerate
torch.set_default_device('cuda')
torch.cuda.device("cuda")
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.set_float32_matmul_precision('high')

In [4]:
model_name = "Qwen/Qwen3-4B"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
print(model.hf_device_map)

{'': 0}


In [7]:
!tw-make tw-simple --rewards dense --goal detailed --seed 1 --test --silent -f --output games/test-game.z8

## Play function

In [8]:
def play(agent, path, max_steps=100, n_episodes=10, verbose=True):
    torch.manual_seed(46)  # For reproducibility when using action sampling.

    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.z8"))

    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=infos_to_request,
                                          max_episode_steps=max_steps)
    env = textworld.gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")

    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(n_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            obs, score, done, infos = env.step(command)
            nb_moves += 1

        agent.act(obs, score, done, infos)  # Let the agent know the game is done.

        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    if verbose:
        if os.path.isdir(path):
            msg = "  \tavg. steps: {:5.1f}; avg. normalized score: {:4.1f} / {}."
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
            print(msg.format(np.mean(avg_moves), np.mean(avg_scores), infos["max_score"]))

## Agents

In [9]:
class RandomAgent(textworld.gym.Agent):
    """ Agent that randomly selects a command from the admissible ones. """
    def __init__(self, seed=1234):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)

    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands=True)

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        return self.rng.choice(infos["admissible_commands"])

In [24]:
class LLMAgent(textworld.gym.Agent):
    """LLM from HuggingFace that acts as an agent."""
    model = None
    tokenizer = None
    context = ""

    token_nothink = "/no_think"
    id_token_close_think = 151668 # </think>
    token_system = "<|im_start|>system\n"
    token_endofturn = "<|im_end|>\n"
    token_user = "<|im_start|>user\n"
    token_assistant = "<|im_start|>assistant\n"
    system_prompt = """
You are an assistant playing a textual game.
The user gives you information on the environment and you reply exclusively in the form \"verb noun\", like \"open box\" or \"take key\".
/no_think
"""
    first_move = False
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.initialize_context()

    def initialize_context(self):
        self.context = self.token_system + self.system_prompt + self.token_endofturn
        self.first_move = True

    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands=True)

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:

        if done:
            self.initialize_context() # resets context
        if self.first_move:
            self.first_move = False
            return "help"
        
        try:
            self.context += self.token_user + obs + self.token_endofturn
            self.context += self.token_assistant # induces model to generate answer
            
            input_ids = self.tokenizer.encode(
                self.context,
                return_tensors = "pt")
            
            generated_ids = self.model.generate(
                input_ids.to("cuda"),
                max_new_tokens = 100,
                eos_token_id = self.tokenizer.eos_token_id
                )
            output_ids = generated_ids[0][len(input_ids[0]):].tolist() 
            
            # parsing thinking content
            try:
                # index finding </think>
                index = len(output_ids) - output_ids[::-1].index(self.id_token_close_think)
            except ValueError:
                index = 0
            response = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
            
            self.context += response + self.token_endofturn

            if len(response.split()) <= 10:
                command = response
            else: # more than 10 words, output is surely wrong
                command = "look"
            return command
            
        except KeyboardInterrupt:
            pass  # Try stopping the game prematurely.

In [35]:
class LLMAgentSelfEvaluate(LLMAgent):
    """LLM from HuggingFace that acts as an agent. It self-evaluates its status and moves."""

    selfeval_turn_counter = 0
    selfeval_turns = 5
    handheld = True
    verbose = False

    def __init__(self, model, tokenizer, selfeval_turns = 5, handheld = True, verbose = False):
        """Initialization function.
        selfeval_turns: how many turns should pass between a self-evaluation and the next one.
        handheld: if this is set to True there are a few simple changes in the function that make it easier for the LLM to understand and correct its course
        """
        super().__init__(model, tokenizer)
        self.selfeval_turns = selfeval_turns
        self.handheld = handheld
        self.verbose = verbose

    def generate_response(self, think=False, max_new_tokens = 30000):
        input_ids = self.tokenizer.encode(
                self.context,
                return_tensors = "pt")
            
        generated_ids = self.model.generate(
            input_ids.to("cuda"),
            max_new_tokens = max_new_tokens,
            eos_token_id = self.tokenizer.eos_token_id
            )
        output_ids = generated_ids[0][len(input_ids[0]):].tolist() 
        
        # parsing thinking content
        try:
            # index finding </think>
            index = len(output_ids) - output_ids[::-1].index(self.id_token_close_think)
        except ValueError:
            index = 0
        if think:
            thinking_response = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
            response = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
            return (thinking_response, response)
        else:
            response = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
            return response

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:

        if done:
            self.initialize_context() # resets context
        if self.first_move and self.handheld:
            self.first_move = False
            return "help"
        if self.selfeval_turn_counter == self.selfeval_turns:
            self.self_evaluation()
            self.selfeval_turn_counter = 0
            obs += self.token_nothink # we re-disable thinking
        
        try:
            self.context += self.token_user + obs + self.token_endofturn
            self.context += self.token_assistant # induces model to generate answer
            
            response = self.generate_response()
            
            self.context += response + self.token_endofturn

            if len(response.split()) <= 10 or not self.handheld:
                command = response
            else: # more than 10 words, output is surely wrong
                command = "look"

            if self.verbose:
                print("GAME ++++++++++++++++++++++++++++++++++++++++++++++++++")
                print(obs)
                print("AGENT -------------------------------------------------")
                print(response)
                if command == "look":
                    print("CHANGED TO: 'look'")

            self.selfeval_turn_counter += 1
            return command
            
        except KeyboardInterrupt:
            pass  # Try stopping the game prematurely.

    def self_evaluation(self):
        self_evaluation_prompt = """
Do you think you're making the right actions in the game? Do you think you're close to reaching the original goal? Think about it. /think
"""
        self.context += self.token_user + self_evaluation_prompt + self.token_endofturn
        self.context += self.token_assistant # induces model to generate answer
        
        (thinking_response, response) = self.generate_response(think=True)
        if self.verbose:
            print(f"SELF-EVALUATION: +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-\n{thinking_response}\n{response}")

    
        

## Run

In [36]:
!tw-make tw-simple --rewards dense --goal detailed --seed 2 --test --silent -f --output games/test-game.z8

In [13]:
max_steps = 100
n_episodes = 1

In [37]:
start = time.time()
play(LLMAgentSelfEvaluate(model, tokenizer, selfeval_turns = 5, handheld=True, verbose=True), "./games/test-game.z8", max_steps=max_steps, n_episodes=n_episodes)
end = time.time()
print(f"Model took {((end - start)/60):.3f} min to play {n_episodes} games")

test-game.z8GAME ++++++++++++++++++++++++++++++++++++++++++++++++++

Available commands:
  look:                describe the current room
  goal:                print the goal of this game
  inventory:           print player's inventory
  go <dir>:            move the player north, east, south or west
  examine ...:         examine something more closely
  eat ...:             eat edible food
  open ...:            open a door or a container
  close ...:           close a door or a container
  drop ...:            drop an object on the floor
  take ...:            take an object that is on the floor
  put ... on ...:      place an object on a supporter
  take ... from ...:   take an object from a container or a supporter
  insert ... into ...: place an object into a container
  lock ... with ...:   lock a door or a container with a key
  unlock ... with ...: unlock a door or a container with a key



>                                                                                     

In [34]:
start = time.time()
play(LLMAgentSelfEvaluate(model, tokenizer, selfeval_turns = 5, handheld=True, verbose=False), "./games/test-game.z8", max_steps=max_steps, n_episodes=n_episodes)
end = time.time()
print(f"Model took {((end - start)/60):.3f} min to play {n_episodes} games")

test-game.z8
Done!

.  	avg. steps: 100.0; avg. score:  3.0 / 8.
Model took 3.098 min to play 1 games
